# Zillow Dataset Cleaning - Phase 1
 Use this as a reference, you **do not** need to run these cells in Phase 1
 - [Link to the orginal dataset](https://www.zillow.com/research/data/) from Zillow's website
 - Select: `ZORI All Homes Plus Multifamily Smoothed` 

In [3]:
# Imports
import pandas as pd
import numpy as np

In [4]:
# Connect to data
FILEPATH = 'https://raw.githubusercontent.com/Lambda-School-Labs/cityspire-a-ds/main/notebooks/Rental_Data/Zillow_Datasets/Zillow_Original.csv'
zillow_original = pd.read_csv(FILEPATH)

In [5]:
# Function to clean up the dataset + yearly average columns

def zillow_cleaner(df):
  ''' A function to convert a dataset from Zillow into something more useable 
  for a predictive model.
  '''

  # Drop columns
  df = df.drop(['RegionID', 'RegionName', 'SizeRank'], axis=1)

  # Change column name
  df = df.rename(columns={"MsaName": "City_State"})

  # Creating yearly average columns
  # 2014 averages
  fourteen_columns = list(df)[1:13]
  df.insert(13, '2014_Average',
                    df[fourteen_columns].mean(axis=1), True)
  # 2015 averages
  fifteen_columns = list(df)[14:26]
  df.insert(26, '2015_Average',
                    df[fifteen_columns].mean(axis=1), True)
  # 2016 averages
  sixteen_columns = list(df)[27:39]
  df.insert(39, '2016_Average',
                    df[sixteen_columns].mean(axis=1), True)
  # 2017 averages
  seventeen_columns = list(df)[40:52]
  df.insert(52, '2017_Average',
                    df[seventeen_columns].mean(axis=1), True)
  # 2018 averages
  eightteen_columns = list(df)[53:65]
  df.insert(65, '2018_Average',
                    df[eightteen_columns].mean(axis=1), True)
  # 2019 averages
  nineteen_columns = list(df)[66:78]
  df.insert(78, '2019_Average',
                    df[nineteen_columns].mean(axis=1), True)
  # 2020 averaes
  twentytwenty_columns = list(df)[79:91]
  df.insert(91, '2020_Average',
                    df[twentytwenty_columns].mean(axis=1), True)

  return df

In [6]:
# Run cleaning function
zillow_original = zillow_cleaner(zillow_original)

# Check it out
print(zillow_original.shape)
zillow_original.head()

(2608, 93)


,City_State,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2020_Average,2021-01
0,"New York, NY",2941.0,2899.0,2913.0,2978.0,2960.0,2954.0,2997.0,3010.0,3016.0,...,3147.0,3075.0,3041.0,2953.0,2909.0,2768.0,2726.0,2703.0,3027.000000,2775.0
1,"Chicago, IL",1419.0,1400.0,1417.0,1445.0,1457.0,1488.0,1489.0,1486.0,1466.0,...,1675.0,1668.0,1636.0,1628.0,1593.0,1575.0,1540.0,1545.0,1626.333333,1571.0
2,"New York, NY",3131.0,3133.0,3126.0,3169.0,3214.0,3284.0,3269.0,3267.0,3261.0,...,3349.0,3303.0,3192.0,3137.0,3038.0,2975.0,2915.0,2851.0,3201.833333,2872.0
3,"Houston, TX",1760.0,1676.0,1741.0,1732.0,1754.0,1768.0,1772.0,1776.0,1787.0,...,1749.0,1754.0,1764.0,1772.0,1778.0,1764.0,1789.0,1798.0,1765.083333,1795.0
4,"Chicago, IL",1615.0,1623.0,1633.0,1658.0,1676.0,1693.0,1718.0,1711.0,1716.0,...,1954.0,1936.0,1920.0,1878.0,1860.0,1815.0,1753.0,1743.0,1880.750000,1721.0


[GeekforGeeks](https://www.geeksforgeeks.org/python-pandas-dataframe-interpolate) - Interpolate Function

In [7]:
# Deal with NaN with interpolate, a way to estimate instead of dropping
zillow_original.interpolate(method='linear', axis=0, inplace=True, limit_direction='both', limit_area='inside', downcast=None)

# Inspect
zillow_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2608 entries, 0 to 2607
Data columns (total 93 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   City_State    2608 non-null   object 
 1   2014-01       2608 non-null   float64
 2   2014-02       2606 non-null   float64
 3   2014-03       2607 non-null   float64
 4   2014-04       2608 non-null   float64
 5   2014-05       2604 non-null   float64
 6   2014-06       2608 non-null   float64
 7   2014-07       2606 non-null   float64
 8   2014-08       2608 non-null   float64
 9   2014-09       2608 non-null   float64
 10  2014-10       2608 non-null   float64
 11  2014-11       2605 non-null   float64
 12  2014-12       2604 non-null   float64
 13  2014_Average  2608 non-null   float64
 14  2015-01       2607 non-null   float64
 15  2015-02       2607 non-null   float64
 16  2015-03       2607 non-null   float64
 17  2015-04       2608 non-null   float64
 18  2015-05       2608 non-null 

Oddly not all of the NaNs were converted to a numerical value based on neighbors. Assumption is that if there are multiple NaNs in sequence the `interpolate()` function has a hard time defining a value.

There were only a hand full of these, so they were corrected manually. Taking the average of the nearest four neighbors, a numerical value was given for the NaN.

# Zillow Dataset Cleaning - Phase 2
- The dataset below has the NaNs correctly manually

In [8]:
# Imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline

In [9]:
# Connect to data
FILEPATH = 'https://raw.githubusercontent.com/Lambda-School-Labs/cityspire-a-ds/main/notebooks/Rental_Data/Zillow_Datasets/Corrected_Zillow.csv'
zillow_corrected = pd.read_csv(FILEPATH)

## Step 1: break apart the state from the city list

This dataset contains multiple cities that are in the same observations. For example *Dallas-Fort Worth*

Thus `explode` is used to seperate the observation into two rows. Giving the model more data to work with.

In order to use this function, the `City/State` columns has to be two seperate columns

[Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html) - *Explode Function*

In [12]:
# Split 'city/state' so the cities can be exploded
zillow_corrected[['City', 'State']] = zillow_corrected['City/State'].str.split(', ', expand=True)
 
# This is being dropped - changing the name later
zillow_corrected = zillow_corrected.drop('City/State', axis=1)
 
# Explode the rows with multiple cities in an observation
zillow_corrected['City'] = zillow_corrected['City'].str.split('-')
zillow_corrected = zillow_corrected.explode('City')

## Step 2: Convert abbreviated state names into full names
- [GitHub](https://gist.githubusercontent.com/rogerallen/1583593/raw/0fffdee6149ab1d993dffa51b1fa9aa466704e18/us_state_abbrev.py) - US State dictionary used below

In [9]:
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'American Samoa': 'AS', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT',
    'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL',
    'Georgia': 'GA', 'Guam': 'GU', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE',
    'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Northern Mariana Islands':'MP', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Puerto Rico': 'PR',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virgin Islands': 'VI', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
 
# Flip the dictonary, need the state codes as keys
us_state_abbrev = {value: key for key, value in us_state_abbrev.items()}
 
# Convert the state codes to state names
zillow_corrected['State'] = zillow_corrected['State'].map(us_state_abbrev)


## Step 3: Re-combine the city and state information

- New column will be `City_State`

In [10]:
# Combine and insert at the front of the dataframe
zillow_corrected.insert(loc=0, column='City_State', 
                        value=(zillow_corrected['City'] + ', ' + 
                               zillow_corrected['State']))
 
# Delete our temp columns
zillow_corrected = zillow_corrected.drop(['City', 'State'], axis=1)

## Step 4: Handle duplicate city-state entries

The original dataset has multiple observations for the same city. The reason behind this is due to a city having multiple zipcodes, thus data was collected by zipcode.

Grouping these observations together into one row is ideal. Taking the average of the combined cities zipcodes helps keep the most data.


In [11]:
# Average accross the city 
zillow_corrected = zillow_corrected.groupby('City_State').mean()

# Round to 2 decimal places
zillow_corrected = zillow_corrected.round(decimals=2)

# Modeling

## Step 1: Separating Columns

The dataset contains columns of months and columns for yearly averages. For the predictive model, the monthly columns will be used. The yearly averages will be used later on.


In [13]:
# Save the averages columns for later
zillow_year_average = zillow_corrected[[col for col in zillow_corrected if 'Average' in col]]

# These non-average columns are for predicting
zillow_corrected = zillow_corrected[[col for col in zillow_corrected if 'Average' not in col]]

## Step 2: Create the Model

In [14]:
X_train = zillow_corrected.iloc[:, :-1]   # Every month but the last month
y_test = zillow_corrected.iloc[:, -1:]    # Only the last month

# Define algorithms for the model
model = make_pipeline(
    OrdinalEncoder(),
    StandardScaler(),
    LinearRegression()
)

# Fit model
model.fit(X_train, y_test)

NameError: name 'make_pipeline' is not defined

## Step 3: Check Metrics

In [ ]:
# New dataframe to display metrics
test_results = pd.DataFrame(index=zillow_corrected.index)

# Original results
test_results['original'] = y_test

# Predicted results from model - rounded
test_results['predicted'] = np.round(model.predict(X_train), 2)

# Take the difference from predicted vs original
test_results['differnce'] = test_results['predicted'] - test_results['original']

# Calculate the percent of error
test_results['percent_error'] = test_results['differnce'].abs() / test_results['original'] * 100

test_results

# Phase 3 - Get Future Predictions

## Step 1: Create a function to help recurse over many months of data

This function serves as a means to repeatedly run the model to extend the dataset as far as needed. It will accept a dataframe, model, and the number of months to generate.

In [ ]:
# Functon to extrapolate from dataset (recursive)
def extrapolate(df, model, n_month_to_predict, remove_negatives=False):
  ''' Takes a dataframe, machine learning model and an integer to represent the
  number of months to be predicted.
  '''
  
  # If we are less than 1, just return the last column, we're done
  if n_month_to_predict < 1:
    return df.iloc[:, -1:]
   
  # How many columns does the model use?
  n_columns = max([s.rank_ for n, s in model.steps if n == 'linearregression'])
   
  # Then make the prediction for this month
  '''
    ':'   -> all rows
    '-n'  -> count form the back 'n' places
    'n:'  -> start at 'n' and go all the way to the end
    '-n:' -> start by counting from the end 'n' places,
               then take from that position to the end
  '''
  df[len(df.columns)] = model.predict(df.iloc[:, -n_columns:])

  # It makes sense for the data, remove negative predictions
  # Set them to zero
  if remove_negatives:
    df[df < 0] = 0

  # Then ask for the next month
  return extrapolate(df, model, n_month_to_predict - 1, remove_negatives)


## Step 2: Generate Data

- 10 years (120 months) of generated data.

In [ ]:
# Make a copy of the dataframe
zillow_corrected_pred = zillow_corrected.copy()

# Extrapolate will tack on "new data" to the dataframe
# zillow_corrected_pred will have all of the predictive data in it
extrapolate(zillow_corrected_pred, model, 120, True)

# Isolate all the predictive data
zillow_corrected_pred = zillow_corrected_pred.iloc[:, len(zillow_corrected.columns):]

Currently the data frame has each month as an integer (84, 85, 86 etc.). These column months need to be changed into an easier to read `YYYY-MM` format. Also each 12 months needs to be averaged out for that year.



In [ ]:
# Yearly averages function
def average_years(df, years):
  ''' Averages the data for specific years and returns as new dataframe
  '''

  yearly_df = pd.DataFrame(index=df.index)
  for y in years:
    yearly_df[y + '_Average'] = np.round(df[[col for col in df if y in col]].mean(axis=1), 2)
    
  return yearly_df


In [ ]:
# Set up columns as dictionary
new_columns = dict()

# Find the first year
first_year = min(list({int(col[:4]) for col in zillow_corrected.columns}))

# Make new associations from integers to `YYYY-MM` formatted strings
for m in zillow_corrected_pred.columns:
   new_columns[m] = str(first_year + int(m / 12)) + '-' + str((m % 12) + 1).zfill(2)

# Replace the column names
zillow_pred = zillow_corrected_pred.rename(columns=new_columns)

# Find all the years in prediction dataset
years = list({col[:4] for col in zillow_pred.columns})
years.sort()

# Save the averages in a new dataframe
zillow_year_pred = average_years(zillow_pred, years)

## Step 3: Export

The data is organized by defualt to have every row be a unique `city_state` and every column to have a specific `year-month`. This won't allow the table to expand easily if a user requests a brand-new year-month. Rearanging the data to have a column for place, time, and price will allow for faster lookups. We can also do this for year averages.

In [ ]:
# Export function 
def export_format(df):
  ''' Make the data into an easily suported shape for the server '''

  # New dataframe
  exportdf = pd.DataFrame(columns = ['City_State', 'Year_Month', 'Price'])
   
  # Break up the original dataframe by rows, and keep track of the index (i)
  # i is the city_state string
  for i, row in df.iterrows():
    # Go through each column of each row
    for col in df.columns:
      # Add a new record to the bottom of the new dataframe
      exportdf.loc[len(exportdf.index)] = [i, col, row[col]]

  return exportdf

In [ ]:
# Convert the historic information we used for the modeling
historic = export_format(zillow_corrected)
historic['Type'] = 'Historic'

# Add the averages that were calculated
historic_avg = export_format(zillow_year_average)
historic_avg['Type'] = 'Calculated'
export = historic.append(historic_avg)

# Convert the predicted information
pred = export_format(zillow_pred)
pred = pred.append(export_format(zillow_year_pred))
pred['Type'] = 'Predicted'

# Mash it all together
export = export.append(pred)

In [ ]:
# Check out export version
export

In [ ]:
# Check out yearly version
zillow_year_pred